# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'

In [60]:
def get_habr_posts(query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title_element = article.find('a', 'tm-title__link')
            title = title_element.text
            relative_link = title_element.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            date_element = article.find('span', class_='tm-article-datetime-published').find('time')
            date = date_element.get('title')
            row = {'date': date, 'title': title, 'link': full_link}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'data science'])
res

,date,title,link
0,"2023-10-24, 13:48",Как устроен GIL (Global Interpreter Lock) в Py...,https://habr.com/ru/companies/otus/articles/76...
1,"2023-10-22, 19:07",Python-разработчик: из новичка в профессионалы...,https://habr.com/ru/companies/productstar/arti...
2,"2020-02-17, 10:34",Go vs Python. Виталий Левченко,https://habr.com/ru/articles/488644/
3,"2023-10-19, 17:25",Решение школьных задач по химии в Python,https://habr.com/ru/articles/767498/
4,"2023-10-23, 17:55",Малоизвестные библиотеки Python для анализа да...,https://habr.com/ru/companies/otus/articles/76...
5,"2023-09-19, 07:42",Microsoft представила дополнение Python Editor...,https://habr.com/ru/news/761862/
6,"2023-10-23, 09:03",Работа с временными рядами в Python. Часть 1,https://habr.com/ru/companies/mvideo/articles/...
7,"2023-08-22, 20:14",Microsoft добавила Python в Excel,https://habr.com/ru/news/756266/
8,"2020-04-21, 18:35","Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
9,"2023-10-09, 08:39",JetBrains и Python Software Foundation рассказ...,https://habr.com/ru/news/766124/


## Дополнительная часть (необязательная)

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
